In [1]:
import torch
import numpy as np
import pinot

Using backend: pytorch


In [2]:
ds = pinot.data.esol()
# ds = pinot.data.utils.batch(ds, 8)
ds_tr, ds_te = pinot.data.utils.split(ds, [2, 1])
ds_tr = pinot.data.utils.batch(ds_tr, len(ds_tr))
ds_te = pinot.data.utils.batch(ds_te, len(ds_te))

In [8]:
graph_kernel = pinot.inference.gp.deep_kernel.DeepKernel(
    representation=pinot.representation.Sequential(
        pinot.representation.dgl_legacy.gn(),
        [32, 'tanh', 32, 'tanh', 32, 'tanh']),
    base_kernel=pinot.inference.gp.kernels.RBF(l=10))

In [9]:
opt = torch.optim.Adam(graph_kernel.parameters(), 1e-3)
x_tr, y_tr = ds_tr[0]
for _ in range(100):
    opt.zero_grad()
    loss = graph_kernel.loss(x_tr, y_tr).sum()
    loss.backward()
    print(loss)
    opt.step()

tensor(1967.9673, grad_fn=<SumBackward0>)
tensor(1920.3394, grad_fn=<SumBackward0>)
tensor(1856.0464, grad_fn=<SumBackward0>)
tensor(1777.9651, grad_fn=<SumBackward0>)
tensor(1694.7557, grad_fn=<SumBackward0>)
tensor(1620.6134, grad_fn=<SumBackward0>)
tensor(1556.6455, grad_fn=<SumBackward0>)
tensor(1485.9333, grad_fn=<SumBackward0>)
tensor(1421.6936, grad_fn=<SumBackward0>)
tensor(1367.1971, grad_fn=<SumBackward0>)
tensor(1312.3926, grad_fn=<SumBackward0>)
tensor(1275.3464, grad_fn=<SumBackward0>)
tensor(1237.4292, grad_fn=<SumBackward0>)
tensor(1209.7474, grad_fn=<SumBackward0>)
tensor(1177.7336, grad_fn=<SumBackward0>)
tensor(1152.2732, grad_fn=<SumBackward0>)
tensor(1123.8618, grad_fn=<SumBackward0>)
tensor(1103.8848, grad_fn=<SumBackward0>)
tensor(1083.0005, grad_fn=<SumBackward0>)
tensor(1067.5665, grad_fn=<SumBackward0>)
tensor(1051.9287, grad_fn=<SumBackward0>)
tensor(1037.5890, grad_fn=<SumBackward0>)
tensor(1024.7435, grad_fn=<SumBackward0>)
tensor(1011.2316, grad_fn=<SumBack

In [11]:
x_te, y_te = ds_te[0]
distribution = graph_kernel.inference(x_tr, y_tr, x_tr)

tensor([[ 9.3572e-02, -1.2454e-03,  2.1827e-04,  ..., -2.2351e-03,
         -3.3247e-04, -8.7970e-04],
        [-1.2410e-03,  3.2186e-02, -7.4506e-05,  ...,  2.1535e-03,
         -4.9162e-04,  2.6702e-03],
        [ 2.1386e-04, -8.5056e-05,  3.4332e-02,  ..., -7.5638e-05,
          8.8847e-04,  6.0558e-05],
        ...,
        [-2.2318e-03,  2.1525e-03, -6.7413e-05,  ...,  4.6346e-02,
         -1.7583e-05,  1.4544e-05],
        [-3.3671e-04, -5.0080e-04,  8.8906e-04,  ..., -2.5153e-05,
          5.2574e-02, -1.5491e-04],
        [-8.7106e-04,  2.6742e-03,  7.4923e-05,  ...,  1.9789e-05,
         -1.4120e-04,  2.4257e-02]], grad_fn=<SubBackward0>)


RuntimeError: cholesky_cpu: U(87,87) is zero, singular U.

In [ ]:
distribution.log_prob(y_te.to(dtype=torch.float32))

In [ ]:
distribution.mean

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt

plt.scatter(y_tr.detach().numpy(), distribution.mean.detach().numpy())